In [4]:
import re
def label_node(ckt_graph,input_vars,key):
    if key in input_vars:
        return 0
    elif ckt_graph[key]["level"] != -1:
        return ckt_graph[key]["level"]+1
    else:
        max_level = -1 
        for input in ckt_graph[key]["inputs"]:
            temp_level = label_node(ckt_graph,input_vars,input)
            if temp_level>max_level:
                max_level = temp_level
        ckt_graph[key]["level"] = max_level
        return max_level+1

def bench2graph(benchfile):
    with open(benchfile, 'r') as file:
        lines = file.readlines()
    
    ckt_graph={}
    output_vars=[]
    input_vars=[]
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        node={}
        if "=" in line:
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            node["level"] = -1
            node["inputs"] = gate_match[2:]
            node["gate"] = gate_match[1]
            ckt_graph[var_name] = node

        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    output_vars.append(match.group(1))
    for key in ckt_graph.keys():
        if key in output_vars:
            label_node(ckt_graph,input_vars,key)
    return ckt_graph,input_vars,output_vars


def graph2bench(ckt_node,inputs,outputs,bench_name):
    bench=[f"# {bench_name}",f"# {len(inputs)} INPUTS", f"# {len(outputs)} OUTPUTS", f"# {len(ckt_node.keys())} Gates", "\n"]

    for input in inputs:
        bench.append(f"INPUT({input})")
    for output in outputs:
        bench.append(f"OUTPUT({output})")
    bench.append("\n")

    gate_no = 0
    key_len = len(ckt_node.keys())
    appending_level = 0
    while gate_no<key_len:
        for key in ckt_node.keys():
            if ckt_node[key]["level"]==appending_level:
                if len(ckt_node[key]["inputs"])>1:
                    gate_inp = ", ".join(ckt_node[key]["inputs"])
                else: gate_inp = ckt_node[key]["inputs"][0]
                gate_name = ckt_node[key]["gate"]
                bench.append(f"{key} = {gate_name}({gate_inp})")
                gate_no += 1
        appending_level += 1

    with open(bench_name+".bench", 'w') as file:
        file.write("\n".join(bench))
        print(bench_name+" bench file created")
                

ckt_node,inputs,outputs=bench2graph("/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c6288/c6288_ttlock_32k.bench")
graph2bench(ckt_node,inputs,outputs,"c6288_ttlock_32k")

c6288_ttlock_32k bench file created


In [7]:
def forward_propagation(logic_cone,ckt_node,inputs,outputs,key):
    if key in outputs:
        pass
    else:
        pass

def label_node(ckt_graph,output_vars,key,level):
    if key in output_vars:
        ckt_graph[key]["level"] = level+1
        ckt_graph[key]["gate"] = "OUTPUT"
        return
    elif ckt_graph[key]["level"] != -1:
        if ckt_graph[key]["level"]<=level:
            ckt_graph[key]["level"]==level+1
        
    else: 
        for input in ckt_graph[key]["inputs"]:
            temp_level = label_node(ckt_graph,input_vars,input)
            if temp_level>max_level:
                max_level = temp_level
        ckt_graph[key]["level"] = max_level
        return max_level+1
    
def logic_cone_forward(benchfile):
    with open(benchfile, 'r') as file:
        lines = file.readlines()
    
    ckt_graph={}
    output_vars=[]
    input_vars=[]
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            if var_name in ckt_graph.keys():
                ckt_graph[var_name]["gate"] = gate_match[1]
            else:  
                node={}  
                node["level"] = -1
                node["outbound"] = []
                node["gate"] = gate_match[1]
                ckt_graph[var_name] = node
            for inp_node in gate_match[2:]:
                if inp_node in ckt_graph.keys():
                    ckt_graph[inp_node]["outbound"].append(var_name)
                else:
                    node={}
                    node["level"] = -1
                    node["outbound"] = [var_name]
                    node["gate"] = ""
                    ckt_graph[inp_node] = node

        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    output_vars.append(match.group(1))
    
    
    for key in ckt_graph.keys():
        if key in input_vars:
            ckt_graph[key]["level"] = 0
            ckt_graph[key]["gate"] = "INPUT"
            label_node(ckt_graph,output_vars,key,0)
    print(ckt_graph)
    """ logic_cone={}
    for key in ckt_node.keys():
        gate_inp = ", ".join(ckt_node[key]["inputs"])
        if "keyinput" in gate_inp:
            logic_cone[key] = ckt_node[key]
            forward_propagation(logic_cone,ckt_node,inputs,outputs,key) """

logic_cone_forward("/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/bench_ckt/c17.bench")

{'G10gat': {'level': -1, 'outbound': ['G22gat'], 'gate': 'NAND'}, 'G1gat': {'level': -1, 'outbound': ['G10gat'], 'gate': ''}, 'G3gat': {'level': -1, 'outbound': ['G10gat', 'G11gat'], 'gate': ''}, 'G11gat': {'level': -1, 'outbound': ['G16gat', 'G19gat'], 'gate': 'NAND'}, 'G6gat': {'level': -1, 'outbound': ['G11gat'], 'gate': ''}, 'G16gat': {'level': -1, 'outbound': ['G22gat', 'G23gat_enc'], 'gate': 'NAND'}, 'G2gat': {'level': -1, 'outbound': ['G16gat'], 'gate': ''}, 'G19gat': {'level': -1, 'outbound': ['G23gat_enc'], 'gate': 'NAND'}, 'G7gat': {'level': -1, 'outbound': ['G19gat'], 'gate': ''}, 'G22gat': {'level': -1, 'outbound': [], 'gate': 'NAND'}, 'G23gat_enc': {'level': -1, 'outbound': [], 'gate': 'NAND'}}


In [ ]:

def convert_gate_line(line, gate_count):
    match = re.findall(r'\b\w+\b', line)
    gate_dict=[ "AND", "NAND", "OR", "NOR", "XOR", "XNOR", "NOT", "BUF", "DFF"]
    
    if match:
        output_var = match[0]
        gate_type = match[1]  # Convert gate type to lowercase
        inputs = match[2:]
        
        if gate_type in gate_dict:
            gate_unique_name = f"{gate_type}{len(inputs)}_{gate_count}"
            converted_line = f"{gate_type.lower()} {gate_unique_name} ({output_var}, {', '.join(inputs)});"
            return gate_type, converted_line
    
    return None, None


def bench2verilog(file_path, file_name):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    gate_count = 1
    input_vars = []
    output_vars = []
    assigned_vars = []
    dff_module = [
            "module DFF(Q, clk, D);",
            "input D;",
            "input clk;",
            "output Q;",
            "always @(clk)", 
            "begin",
            "  Q <= D;", 
            "end", 
            "endmodule"]
    opr_lines=[]
    gate_types=[]

    # Regular expressions to match INPUT, OUTPUT, and gate assignments
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    assign_pattern = re.compile(r'^(\w+)\s*=\s*(NAND|NOR|AND|OR|XOR|XNOR|NOT|DFF|BUF)\((.*)\)')

    for line in lines:
        line = line.strip()
        if line.startswith("INPUT"):
            match = input_pattern.match(line)
            if match:
                input_vars.append(match.group(1))
        elif line.startswith("OUTPUT"):
            match = output_pattern.match(line)
            if match:
                output_vars.append(match.group(1))
        else:
            match = assign_pattern.match(line)
            if match:
                assigned_vars.append(match.group(1))
                gate_type,opr_line = convert_gate_line(line,gate_count)
                if opr_line is not None:
                    opr_lines.append(opr_line)
                    if gate_type not in gate_types:
                        gate_types.append(gate_type) 
                    gate_count += 1

    # Intermediate variables are those assigned but not inputs or outputs
    intermediate_vars = [var for var in assigned_vars if var not in input_vars and var not in output_vars]

    module_heading = f"module {file_name}({','.join(input_vars + output_vars)});"
    input_line = f"input {','.join(input_vars)};"
    output_line = f"output {','.join(output_vars)};"
    wire_line = f"wire {','.join(intermediate_vars)};"
    termination_line = "endmodule"
    main_module="\n".join([module_heading,"",input_line,output_line,wire_line,""]+opr_lines+["",termination_line])
    
    if "DFF" in gate_types:
        main_module = "\n".join(dff_module + ["\n", main_module])
    return main_module

